In [ ]:
import pandas as pd
import numpy as np

In [ ]:
"""
NOTE: combining 'ORI' and 'Incident Number' is not enough for a unique identifier per event. 
Waiting to hear clarification form Ben, but could create a key using date also. That should
be enough to join matching entries from one table to another correctly. 
"""

In [ ]:
# Deliverables 1 & 2 Need to join:
# Data/2020 Domestic Violence - Administrative Data.csv
# Data/2020 Domestic Violence - Other Data.csv
# Data/2020 Q2 Domestic Violence - Location Type.csv
admin_df = pd.read_csv('../Data/2020 Domestic Violence - Administrative Data.csv', encoding='ISO-8859-1',skiprows=7)
other_df = pd.read_csv('../Data/2020 Domestic Violence - Other Data.csv', encoding='ISO-8859-1',skiprows=7)
location_df = pd.read_csv('../Data/2020 Q1 Domestic Violence - Location Type.csv', encoding='ISO-8859-1',skiprows=6)


In [ ]:
print('Lengths: Admin | Other | Locations\n',len(admin_df), len(other_df), len(location_df))
print('W/o true duplicates\n', len(admin_df.drop_duplicates()), len(other_df.drop_duplicates()), len(location_df.drop_duplicates()))

In [ ]:
# Verifying that not all duplicate Incident Number cases are complete duplicates 
print('Admin Dropped complete duplicates: ',len(admin_df.drop_duplicates()),'\nAdmin Dropped duplicate incident numbers: ', len(admin_df.drop_duplicates('Incident Number')))

In [ ]:
admin_df['Id_Key'] = admin_df['ORI'].values + ' - ' + admin_df['Incident Number'].values
print('Admin Dropped Duplicates: ', len(admin_df.drop_duplicates()), '\nAdmin Dropped Id_Key dups: ', len(admin_df.drop_duplicates('Id_Key')))

In [ ]:
# Trying to figure out what is happenning in the duplicate Id_Key that aren't picked up as true duplicates
# In doing so are we dropping acutal repeated entries or seperate events? 
dd_ind = set(admin_df.drop_duplicates().index)
key_ind = set(admin_df.drop_duplicates('Id_Key').index)
print(len(dd_ind), len(key_ind))
dif_ind = dd_ind.difference(key_ind)

In [ ]:
admin_df.iloc[1023], admin_df.iloc[1024]

In [ ]:
for i in range(len(admin_df.drop_duplicates())):
    X = admin_df.iloc[i]
    ori, incnum = X.ORI, X['Incident Number']
    Y = other_df.loc[other_df.ORI == ori]
    Z = Y[Y['Incident Number'] == incnum]
    break

In [ ]:
Z

In [ ]:
test = other_df.loc[other_df.ORI == 'TN0540100']
test = test[test['Incident Number'] == '2.02E+11']
test